In [1]:
'''
Pull price data from Marketstack API
Created JR 09/10/2024
Requires API key from Marketstack. 
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 
           'FB', 'GOOGL', 'AVGO', 'BRK-B', 
           'GOOG', 'LLY', 'TSLA', 'JPM', 
           'UNH', 'XOM', 'V']
end_date = "2024-09-30"  
start_date = "2019-09-30"  

# API EOD URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Function to fetch EOD data for a single symbol
def get_eod_data(symbol, start_date, end_date, api_key):
    
    all_data = []  
    limit = 1000  # confirmed max 1000 via marketstack docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbol,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit, # required - if this is missing only returns the first 1000. 
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data. 
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    df = pd.DataFrame(all_data)
    df['symbol'] = symbol  # Add a column for the symbol

    return df

# Initialize an empty DataFrame to store all the data
price_df = pd.DataFrame()

# Loop through each symbol and fetch its data
for symbol in symbols:
    symbol_df = get_eod_data(symbol, start_date, end_date, API_KEY)
    price_df = pd.concat([price_df, symbol_df], ignore_index=True)

print(price_df.head())

# Format the date
price_df = price_df[['date', 'adj_close', 'symbol']]
price_df['date'] = pd.to_datetime(price_df['date']).dt.date

# Switch to array with symbols 
price_df = price_df.pivot(index='date', columns='symbol', values='adj_close')

print(price_df.head())

# Save to csv to prevent having to make multiple calls whilst testing
price_df.to_csv('combined_eod_data.csv', index=True)

      open    high      low   close      volume  adj_high  adj_low  adj_close  \
0  230.155  233.00  229.655  233.00  52779736.0    233.00   229.65     233.00   
1  228.460  229.52  227.300  227.79  33993600.0    229.52   227.30     227.79   
2  227.300  228.50  225.410  227.52  36490267.0    228.50   225.41     227.52   
3  224.930  227.29  224.030  226.37  40779133.0    227.29   224.02     226.37   
4  228.645  229.35  225.740  227.37  41781114.0    229.35   225.73     227.37   

   adj_open  adj_volume  split_factor  dividend symbol exchange  \
0   230.040  54793391.0           1.0       0.0   AAPL     XNAS   
1   228.460  34025967.0           1.0       0.0   AAPL     XNAS   
2   227.300  36636707.0           1.0       0.0   AAPL     XNAS   
3   224.930  42308715.0           1.0       0.0   AAPL     XNAS   
4   228.645  43556068.0           1.0       0.0   AAPL     XNAS   

                       date  
0  2024-09-30T00:00:00+0000  
1  2024-09-27T00:00:00+0000  
2  2024-09-26T00:00: